In [2]:
# Import Libraries
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import os

hf_token = os.getenv('HF_TOKEN')
login(token=hf_token, add_to_git_credential=True)

In [3]:
# Instruct Models
LLAMA_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3_MODEL = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2_MODEL = "google/gemma-2-2b-it"
QWEN2_MODEL = "Qwen/Qwen2-7B-Instruct"

messages = [
    {"role": "system", "content": "You are a helpful AI Assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Chess Players"}
]

# Quantization Config - Load model into memory and use less memory
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [4]:
def generate_response(MODEL, MESSAGES):
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=MODEL)
    tokenizer.pad_token = tokenizer.eos_token
    model_inputs = tokenizer.apply_chat_template(conversation=MESSAGES, return_tensors="pt", add_generation_prompt=True).to("cuda")

    # Stream Responses
    streamer = TextStreamer(tokenizer=tokenizer)
    model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=MODEL, device_map="auto", quantization_config=quant_config)
    model_output = model.generate(model_inputs, max_new_tokens=80, streamer=streamer)
    del tokenizer, streamer, model, model_inputs, model_output
    torch.cuda.empty_cache()

In [5]:
# LLAMA MODEL
generate_response(MODEL=LLAMA_MODEL, MESSAGES=messages)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful AI Assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Chess Players<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Why did the pawn go to therapy? 

Because it was feeling a little "moved" and had a lot of "check" issues to work through.<|eot_id|>


In [6]:
# PHI3 MODEL
generate_response(MODEL=PHI3_MODEL, MESSAGES=messages)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

<|system|> You are a helpful AI Assistant<|end|><|user|> Tell a light-hearted joke for a room of Chess Players<|end|><|assistant|> Why did the chess player get kicked out of the bar? Because he was always making a move too fast and never left his pieces behind!<|end|>


In [7]:
# QWEN2 MODEL
generate_response(MODEL=QWEN2_MODEL, MESSAGES=messages)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

<|im_start|>system
You are a helpful AI Assistant<|im_end|>
<|im_start|>user
Tell a light-hearted joke for a room of Chess Players<|im_end|>
<|im_start|>assistant
Sure, here's a light-hearted chess joke for the room:

Why did the chess piece refuse to go on a date with the queen?

Because it had heard about her check-mates!<|im_end|>


In [8]:
# GEMMA MODEL
messages.pop(0)
generate_response(MODEL=GEMMA2_MODEL, MESSAGES=messages)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

<bos><start_of_turn>user
Tell a light-hearted joke for a room of Chess Players<end_of_turn>
<start_of_turn>model


The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


Why did the chess player bring a ladder to the game? 

Because they heard the opponent was playing at a higher level! ♟️🪜 


Let me know if you'd like to hear another one! 😊 
<end_of_turn>
